ข้อ 1

In [ ]:
SELECT dir_fname, dir_lname, gen_title
FROM Movie
JOIN movie_direction USING(mov_id)
JOIN director USING(dir_id)
NATURAL JOIN rating
NATURAL JOIN movie_genres
NATURAL JOIN genres
GROUP BY dir_fname, dir_lname, gen_title
HAVING MAX(rev_stars) = MAX(rev_stars);

ข้อ 2

In [ ]:
SELECT actor.act_fname, actor.act_lname,actor.act_gender,SUM(movie.mov_time) AS total_time
FROM movie
JOIN movie_cast USING(mov_id)
JOIN actor USING(act_id)
LEFT JOIN rating USING(mov_id)
WHERE rating.rev_stars IS NOT NULL
GROUP BY actor.act_fname, actor.act_lname,actor.act_gender;

ข้อ 3

In [ ]:
SELECT director.dir_fname,director.dir_lname,actor.act_fname,actor.act_lname,COUNT(*) AS collaboration_count
FROM movie
JOIN movie_cast ON movie.mov_id = movie_cast.mov_id
JOIN actor ON movie_cast.act_id = actor.act_id
JOIN movie_direction ON movie.mov_id = movie_direction.mov_id
JOIN director ON movie_direction.dir_id = director.dir_id
WHERE actor.act_gender = 'F'
GROUP BY director.dir_id, actor.act_id
ORDER BY collaboration_count DESC
LIMIT 5;

ข้อ 4

In [ ]:
ALTER TABLE genres
ADD COLUMN gen_id_temp INTEGER;

-- อัปเดตคอลัมน์ชั่วคราวด้วยค่าตำแหน่งของ gen_title
UPDATE genres
SET gen_id_temp = ROW_NUMBER() OVER (ORDER BY gen_title); #แก้เรียงจากน้อยไปมาก

-- ลบคอลัมน์ gen_id เดิม
ALTER TABLE genres
DROP COLUMN gen_id;

-- แล้วเปลี่ยนชื่อคอลัมน์ชั่วคราวเป็น gen_id
ALTER TABLE genres
RENAME COLUMN gen_id_temp TO gen_id;

ข้อ 5

In [ ]:
UPDATE actor
SET act_gender = 'F'
WHERE 
  UPPER(act_fname) LIKE 'EM%'
  OR UPPER(act_fname) LIKE 'CHAR%'
  OR UPPER(act_lname) LIKE '%DY'
  OR UPPER(act_lname) LIKE '%SY'
  OR UPPER(act_lname) LIKE '%LIA';